# Answering Business Questions Using SQL

## Introduction 

This project will answer various business questions using SQL.

The database used is called Chinook, which is a sample database that represents a digital media store, including tables for artists, albums, media tracks, invoices, and customers.

* Media-related data was created using real data from an Apple iTunes library.
* Customer and employee information was created using fictitious names and addresses that can be located on Google maps, and other well formatted data (phone, fax, email, etc.)
* Sales information was auto generated using random data for a four year period.

The Chinook sample database includes:

* 11 tables
* A variety of indexes, primary and foreign key constraints
* Over 15,000 rows of data

A diagram of the databse schema can be found [here](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg).

Connecting the Jupyter Notebook to the database file:

In [4]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

Quering the database to get a list of all the tables and views in our database:

In [5]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table", "view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting New Albums to Purchase

Hypothetical Scenario:

The Chinook record store has just signed a deal with a new record label, and I've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

Artist Name and Genre
* Regal / Hip-Hop
* Red Tone / Punk
* Meteor and the Girls / Pop
* Slim Jim Bites / Blues

The record label specialises in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so I'm interested in finding out which genres sell the best in the USA.

In [ ]:
%% sql

